0. imports

In [ ]:
import numpy as np
import pandas as pd
import os, time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import models
import torch.nn as nn
from torch import optim
import torch
from timeit import default_timer as timer
from torch import cuda
from torch.utils.data import DataLoader
from keras.preprocessing.image import img_to_array, array_to_img
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import cv2

In [ ]:
!pip install wandb -q

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: tars. Use `wandb login --relogin` to force relogin


True

1. dataset

In [ ]:
! gdown --id 1KlHddR1fMXyzzDoEJ0zIZEnEjwOQh3uN

In [ ]:
# Read csv data files
train_data = pd.read_csv('/content/fashion-mnist_train.csv')
test_data = pd.read_csv('/content/fashion-mnist_test.csv')

In [ ]:
train_data.shape #(60,000*785)
test_data.shape #(10000,785)
train_X= np.array(train_data.iloc[:,1:])
test_X= np.array(test_data.iloc[:,1:])
train_Y= np.array (train_data.iloc[:,0]) # (60000,)
test_Y = np.array(test_data.iloc[:,0]) #(10000,)

In [ ]:
# Convert the images into 3 channels
train_X=np.dstack([train_X] * 3)
test_X=np.dstack([test_X]*3)
train_X.shape

In [ ]:
# Reshape images as per the tensor format required by tensorflow
train_X = train_X.reshape(-1, 28,28,3)
test_X= test_X.reshape (-1,28,28,3)
train_X.shape

In [ ]:
# Resize the images 48*48 as required by VGG16
from keras.preprocessing.image import img_to_array, array_to_img
train_X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in train_X])
test_X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in test_X])
#train_x = preprocess_input(x)
train_X.shape

In [ ]:
# Normalise the data and change data type
train_X = train_X / 255.
test_X = test_X / 255.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

In [ ]:
# Splitting train data as train and validation data
train_X,valid_X,train_label,valid_label = train_test_split(train_X,
                                                           train_Y,
                                                           test_size=0.2,
                                                           random_state=13
                                                           )

In [ ]:
#train_X = preprocess_input(train_X)
#valid_X = preprocess_input(valid_X)
#test_X  = preprocess_input (test_X)

2. vgg16 model

In [ ]:
wandb.init(name='test_run_4',
           project='test_project_4',
           entity='tars')

In [ ]:
model = models.vgg16(pretrained=True)

In [ ]:
for param in model.parameters():
  param.requires_grad = False

In [ ]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
#model.classifier[6].out_features=10

In [ ]:
n_inputs = model.classifier[6].in_features
"""model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 10),
                      nn.LogSoftmax(dim=1))"""
model.classifier[6] = nn.Linear(in_features=n_inputs, out_features=10, bias=True)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

134,301,514 total parameters.
40,970 training parameters.


In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
tensor_x = torch.Tensor(train_X.copy()) # transform to torch tensor
tensor_y = torch.Tensor(train_label)

my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset, batch_size=128, shuffle=True) # create your dataloader

In [ ]:
wandb.watch(model)

[]

In [ ]:
tensor_x1 = torch.Tensor(valid_X.copy()) # transform to torch tensor
tensor_y1 = torch.Tensor(valid_label)

my_dataset1 = TensorDataset(tensor_x1,tensor_y1) # create your datset
my_dataloader1 = DataLoader(my_dataset1, batch_size=128) # create your dataloader

In [ ]:
for epoch in range(0, 10):
  train_loss = 0.0
  valid_loss = 0.0
  train_acc = 0
  valid_acc = 0

  for data, targets in my_dataloader:
    # Generate predictions
    data1 = data.cpu().detach().numpy()
    data2 = np.zeros((128,224,224,3))
    for i in range(0, 128):
      data2[i] = cv2.resize(data1[i], dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    data = torch.Tensor(data2.copy())

    data = data.permute(0, 3, 1, 2)
    out = model(data)
    # Calculate loss
    targets = targets.to(torch.long)
    loss = criterion(out, targets)
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()
    # train loss
    train_loss += loss.item() * data.size(0)
    # train accuracy
    _, pred = torch.max(out, dim=1)
    correct_tensor = pred.eq(targets.data.view_as(pred))
    accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
    train_acc += accuracy.item() * data.size(0)

  # val metrix
  for data, targets in my_dataloader1:
    # Generate predictions
    data1 = data.cpu().detach().numpy()
    data2 = np.zeros((128,224,224,3))
    for i in range(0, 128):
      data2[i] = cv2.resize(data1[i], dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    data = torch.Tensor(data2.copy())

    data = data.permute(0, 3, 1, 2)
    out = model(data)

    # loss
    targets = targets.to(torch.long)
    loss = criterion(out, targets)

    # validation loss
    valid_loss += loss.item() * data.size(0)

    # Calculate validation accuracy
    _, pred = torch.max(out, dim=1)
    correct_tensor = pred.eq(targets.data.view_as(pred))
    accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
    valid_acc += accuracy.item() * data.size(0)

  # Calculate average losses
  train_loss = train_loss / len(my_dataloader.dataset)
  valid_loss = valid_loss / len(my_dataloader1.dataset)
  # Calculate average accuracy
  train_acc = train_acc / len(my_dataloader.dataset)
  valid_acc = valid_acc / len(my_dataloader1.dataset)

  wandb.log({"Epoch": epoch,
           "Train Loss": train_loss,
           "Train Acc": train_acc,
           "Valid Loss": valid_loss,
           "Valid Acc": valid_acc})
  wandb.watch(model)

In [ ]:
torch.save(model.state_dict(), "/content/model_with5epochs.pt")

In [ ]:
! gdown --id 1_Xq2rppZdAC_45cX8iBxXHFdXViw282-

In [ ]:
model1 = models.vgg16(pretrained=True)
for param in model1.parameters():
  param.requires_grad = False
n_inputs = model1.classifier[6].in_features
model1.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 10),
                      nn.LogSoftmax(dim=1))
model1.load_state_dict(torch.load("/content/model_with5epochs.pt"))
model1.eval()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
tensor_xt = torch.Tensor(test_X.copy()) # transform to torch tensor
tensor_yt = torch.Tensor(test_Y)

my_datasett = TensorDataset(tensor_xt,tensor_yt) # create your datset
my_dataloadert = DataLoader(my_datasett, batch_size=1) # create your dataloader

In [ ]:
y_pred = {}
count = 0
for data, targets in my_dataloadert:
  # Generate predictions
  if (count%1000==0):
    print(count)
  data = data.permute(0, 3, 1, 2)
  out = model1(data)
  _, pred = torch.max(out, dim=1)
  y_pred[count] = pred
  count += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
pred = list(y_pred.values())